In [1]:
#%matplotlib notebook

In [2]:
#List dependencies: we used commas instead of listing them out individually on seperate lines to save space.
import csv, os, requests, time, pandas as pd, matplotlib.pyplot as plt, numpy as np
#using the * imports everything in the config file, where the API key for Yelp (ykey) is stored
from config import *

In [3]:
#
yelpapi_csv = os.path.join("yelpapidata.csv")
yelp_raw = pd.read_csv(yelpapi_csv)
yelp_raw.head()

,search_city,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph
0,"Montgomery, Alabama",American,BhPj9fLZph1TcofvuTZT1w,Urban Cookhouse,https://s3-media1.fl.yelpcdn.com/bphoto/XY8xxa...,https://www.yelp.com/biz/urban-cookhouse-montg...,False,142,4.5,$,32.360651,-86.157520,7712 Eastchase Pkwy,Montgomery,AL,36117,US,(334) 239-7801
1,"Montgomery, Alabama",American,4uVYV49MDmm3tunRg6hnew,Stockyard Grill,https://s3-media2.fl.yelpcdn.com/bphoto/gfK2DK...,https://www.yelp.com/biz/stockyard-grill-montg...,False,51,4.5,$,32.317361,-86.350667,4500 Mobile Hwy,Montgomery,AL,36108,US,(334) 288-2575
2,"Montgomery, Alabama",American,s2uIZz572lvzeeaT40xm5g,Plantation House and Garden,https://s3-media3.fl.yelpcdn.com/bphoto/E4UqZ2...,https://www.yelp.com/biz/plantation-house-and-...,False,14,4.0,$$,32.479502,-86.369687,3240 Grandview Rd,Millbrook,AL,36054,US,(334) 285-1466
3,"Montgomery, Alabama",American,TmPADgXsze5_gfTZPZ24vQ,Southern Comfort,https://s3-media2.fl.yelpcdn.com/bphoto/3WRpNu...,https://www.yelp.com/biz/southern-comfort-hope...,False,193,4.5,$$,32.269824,-86.363997,210 Wasden Rd,Hope Hull,AL,36043,US,(334) 280-0012
4,"Montgomery, Alabama",American,u-pZS8ZvEm8QwJIMrf1Wbw,Common Bond Brewers,https://s3-media3.fl.yelpcdn.com/bphoto/xNl0HP...,https://www.yelp.com/biz/common-bond-brewers-m...,False,16,5.0,$,32.377410,-86.313590,424 Bibb St,Montgomery,AL,36104,US,(334) 676-2287


In [4]:
yelp_raw.describe()

,review_count,rating,lat,long
count,27851.000000,27851.000000,27850.000000,27850.000000
mean,177.127679,3.967398,38.425217,-92.894844
std,292.762941,0.604082,5.160674,18.784798
min,1.000000,1.000000,21.262370,-158.105524
25%,30.000000,3.500000,35.493787,-104.917115
50%,88.000000,4.000000,39.667450,-89.220339
75%,212.000000,4.500000,41.822154,-78.533204
max,9460.000000,5.000000,58.398782,157.941630


In [5]:
yelp_format_rd1 = yelp_raw.copy()
#yelp_format_rd1["review_count"] = yelp_format_rd1.loc[:, "review_count"].astype(float)
#yelp_format_rd1["rating"] = yelp_format_rd1.loc[:, "rating"].astype(float)
yelp_format_rd1['price_attribute'] = yelp_format_rd1['price'].replace(
    {'$': '1.Affordable', '$$': '2.Moderate', '$$$': '3.Expensive'})
yelp_format_rd1.head()

,search_city,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph,price_attribute
0,"Montgomery, Alabama",American,BhPj9fLZph1TcofvuTZT1w,Urban Cookhouse,https://s3-media1.fl.yelpcdn.com/bphoto/XY8xxa...,https://www.yelp.com/biz/urban-cookhouse-montg...,False,142,4.5,$,32.360651,-86.157520,7712 Eastchase Pkwy,Montgomery,AL,36117,US,(334) 239-7801,1.Affordable
1,"Montgomery, Alabama",American,4uVYV49MDmm3tunRg6hnew,Stockyard Grill,https://s3-media2.fl.yelpcdn.com/bphoto/gfK2DK...,https://www.yelp.com/biz/stockyard-grill-montg...,False,51,4.5,$,32.317361,-86.350667,4500 Mobile Hwy,Montgomery,AL,36108,US,(334) 288-2575,1.Affordable
2,"Montgomery, Alabama",American,s2uIZz572lvzeeaT40xm5g,Plantation House and Garden,https://s3-media3.fl.yelpcdn.com/bphoto/E4UqZ2...,https://www.yelp.com/biz/plantation-house-and-...,False,14,4.0,$$,32.479502,-86.369687,3240 Grandview Rd,Millbrook,AL,36054,US,(334) 285-1466,2.Moderate
3,"Montgomery, Alabama",American,TmPADgXsze5_gfTZPZ24vQ,Southern Comfort,https://s3-media2.fl.yelpcdn.com/bphoto/3WRpNu...,https://www.yelp.com/biz/southern-comfort-hope...,False,193,4.5,$$,32.269824,-86.363997,210 Wasden Rd,Hope Hull,AL,36043,US,(334) 280-0012,2.Moderate
4,"Montgomery, Alabama",American,u-pZS8ZvEm8QwJIMrf1Wbw,Common Bond Brewers,https://s3-media3.fl.yelpcdn.com/bphoto/xNl0HP...,https://www.yelp.com/biz/common-bond-brewers-m...,False,16,5.0,$,32.377410,-86.313590,424 Bibb St,Montgomery,AL,36104,US,(334) 676-2287,1.Affordable


In [6]:
yelp_format_rd1.columns

Index(['search_city', 'search_cuisine', 'y_id', 'name', 'image_url', 'url',
       'closed', 'review_count', 'rating', 'price', 'lat', 'long', 'add1',
       'city', 'state', 'zip_code', 'country', 'disp_ph', 'price_attribute'],
      dtype='object')

In [7]:
yelp_format_rd1.count()

search_city        27851
search_cuisine     27851
y_id               27851
name               27851
image_url          27532
url                27851
closed             27851
review_count       27851
rating             27851
price              23729
lat                27850
long               27850
add1               27496
city               27851
state              27851
zip_code           27822
country            27851
disp_ph            27251
price_attribute    23729
dtype: int64

In [8]:
yelp_format_rd1.dtypes

search_city         object
search_cuisine      object
y_id                object
name                object
image_url           object
url                 object
closed                bool
review_count         int64
rating             float64
price               object
lat                float64
long               float64
add1                object
city                object
state               object
zip_code            object
country             object
disp_ph             object
price_attribute     object
dtype: object

In [9]:
yelp_gb_cities = yelp_format_rd1.groupby(['search_city'])
yelp_gb_cities.sum().head()

,closed,review_count,rating,lat,long
search_city,,,,,
"Albany, New York",False,61218,2506.0,27242.467834,-47077.268325
"Annapolis, Maryland",False,96385,1912.0,19436.953223,-38130.413818
"Atlanta, Georgia",False,287240,3898.0,32554.059412,-81150.804242
"Augusta, Maine",False,15429,1021.0,11696.821371,-18341.337151
"Austin, Texas",False,299061,3792.0,27810.980645,-89635.319822


In [10]:
yelp_gb_cuisines = yelp_format_rd1.groupby(['search_cuisine'])
yelp_gb_cuisines.sum().head()

,closed,review_count,rating,lat,long
search_cuisine,,,,,
American,0.0,658067,9574.5,93656.108174,-222832.643590
Cajun,0.0,364110,5698.0,56090.625394,-139019.652525
Caribbean,0.0,89119,3081.5,29446.371743,-67662.051855
Chinese,0.0,215002,7071.0,73497.901854,-178476.610595
French,0.0,251601,4517.5,43102.898712,-105380.021478


In [11]:
yelp_gb_price = yelp_format_rd1.groupby(['price'])
yelp_gb_price.sum().head()

,closed,review_count,rating,lat,long
price,,,,,
$,2.0,736368,26193.5,257384.932815,-6.172942e+05
$$,0.0,3755842,62684.0,611843.526283,-1.491098e+06
$$$,0.0,334326,4027.0,38330.086618,-9.183231e+04
$$$$,0.0,30459,460.0,3951.409992,-1.070779e+04


In [12]:
yelp_gb_citiescount = yelp_format_rd1.groupby(['search_city'])
yelp_gb_citiescount.count().head()

,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph,price_attribute
search_city,,,,,,,,,,,,,,,,,,
"Albany, New York",638,638,638,628,638,638,638,638,531,638,638,632,638,638,638,638,622,531
"Annapolis, Maryland",498,498,498,496,498,498,498,498,437,498,498,490,498,498,498,498,493,437
"Atlanta, Georgia",962,962,962,960,962,962,962,962,851,962,962,960,962,962,962,962,947,851
"Augusta, Maine",264,264,264,253,264,264,264,264,210,264,264,264,264,264,264,264,261,210
"Austin, Texas",917,917,917,915,917,917,917,917,768,917,917,910,917,917,916,917,890,768


In [13]:
#Create a heatmap and show the concentration of restaurants in each capitol city
! jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [14]:
!jupyter nbextension list

Known nbextensions:
  config dir: C:\Users\Nell\.jupyter\nbconfig
    notebook section
      jupyter-gmaps/extension enabled 
  config dir: C:\Users\Nell\Anaconda3\etc\jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 


      - Validating: ok
      - Validating: ok


In [15]:
%matplotlib inline

In [16]:
#Bring in dependencies, config files, and configure the gmaps
import gmaps
import pandas as pd

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [17]:
#import pandas as pd
import pandas as pd

#calling DataFrame constructor
df = pd.DataFrame()

print(df)

Empty DataFrame
Columns: []
Index: []


In [18]:
#Create yelp dataframe to compare concentration of businesses in cities
cities_df = pd.read_csv('cities.csv')
cities_df.dropna()
cities_df.head()

,Montgomery,Alabama
0,Juneau,Alaska
1,Phoenix,Arizona
2,Little Rock,Arkansas
3,Sacramento,California
4,Denver,Colorado


In [19]:
#create a yelp dataframe for just cities, lat, long, and name of business
yelp_raw[['search_city', 'lat', 'long', 'name']]

,search_city,lat,long,name
0,"Montgomery, Alabama",32.360651,-86.157520,Urban Cookhouse
1,"Montgomery, Alabama",32.317361,-86.350667,Stockyard Grill
2,"Montgomery, Alabama",32.479502,-86.369687,Plantation House and Garden
3,"Montgomery, Alabama",32.269824,-86.363997,Southern Comfort
4,"Montgomery, Alabama",32.377410,-86.313590,Common Bond Brewers
...,...,...,...,...
27846,"Cheyenne, Wyoming",40.657480,-105.160399,Empire Express
27847,"Cheyenne, Wyoming",40.578299,-105.077356,College Cafe
27848,"Cheyenne, Wyoming",40.574059,-105.100014,Petra Grill Mediterranean Cuisine
27849,"Cheyenne, Wyoming",40.574839,-105.099282,Tikka Grill


In [20]:
import gmaps
from config import gkey

fig = gmaps.figure()
fig


Figure(layout=FigureLayout(height='420px'))

In [21]:
#Create a data frame for a specific city, i.e. Phoenix, Arizona.  Show the number of ratings and the coordinates
specific_city = 'Phoenix, Arizona'
yelp_gmaps = yelp_format_rd1[yelp_format_rd1['search_city'] == specific_city]
yelp_gmaps = yelp_gmaps[['lat','long','review_count']]
yelp_gmaps.head()

,lat,long,review_count
476,33.460210,-112.070420,20
477,33.458812,-112.070364,601
478,33.509031,-112.071295,1677
479,33.447392,-112.072831,2393
480,33.560850,-112.116190,3357


In [22]:
#create a yelp dataframe for just cities, lat, long, and price
yelp_raw[['search_city', 'lat', 'long', 'price']].dropna()
yelp_raw['price_level'] = yelp_raw['price'].replace(
    {'$': 1, '$$': 2, '$$$': 3})
yelp_raw[['search_city', 'price_level', 'lat', 'long']].dropna()
yelp_raw

,search_city,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph,price_level
0,"Montgomery, Alabama",American,BhPj9fLZph1TcofvuTZT1w,Urban Cookhouse,https://s3-media1.fl.yelpcdn.com/bphoto/XY8xxa...,https://www.yelp.com/biz/urban-cookhouse-montg...,False,142,4.5,$,32.360651,-86.157520,7712 Eastchase Pkwy,Montgomery,AL,36117,US,(334) 239-7801,1
1,"Montgomery, Alabama",American,4uVYV49MDmm3tunRg6hnew,Stockyard Grill,https://s3-media2.fl.yelpcdn.com/bphoto/gfK2DK...,https://www.yelp.com/biz/stockyard-grill-montg...,False,51,4.5,$,32.317361,-86.350667,4500 Mobile Hwy,Montgomery,AL,36108,US,(334) 288-2575,1
2,"Montgomery, Alabama",American,s2uIZz572lvzeeaT40xm5g,Plantation House and Garden,https://s3-media3.fl.yelpcdn.com/bphoto/E4UqZ2...,https://www.yelp.com/biz/plantation-house-and-...,False,14,4.0,$$,32.479502,-86.369687,3240 Grandview Rd,Millbrook,AL,36054,US,(334) 285-1466,2
3,"Montgomery, Alabama",American,TmPADgXsze5_gfTZPZ24vQ,Southern Comfort,https://s3-media2.fl.yelpcdn.com/bphoto/3WRpNu...,https://www.yelp.com/biz/southern-comfort-hope...,False,193,4.5,$$,32.269824,-86.363997,210 Wasden Rd,Hope Hull,AL,36043,US,(334) 280-0012,2
4,"Montgomery, Alabama",American,u-pZS8ZvEm8QwJIMrf1Wbw,Common Bond Brewers,https://s3-media3.fl.yelpcdn.com/bphoto/xNl0HP...,https://www.yelp.com/biz/common-bond-brewers-m...,False,16,5.0,$,32.377410,-86.313590,424 Bibb St,Montgomery,AL,36104,US,(334) 676-2287,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27846,"Cheyenne, Wyoming",Vietnamese,Xr_UtPKedDyleG_MSHvK-Q,Empire Express,https://s3-media1.fl.yelpcdn.com/bphoto/00xiho...,https://www.yelp.com/biz/empire-express-laport...,False,32,2.5,$,40.657480,-105.160399,3300 W County Rd,Laporte,CO,80535,US,(970) 493-2663,1
27847,"Cheyenne, Wyoming",Vietnamese,FS9UR3iy80xk4CityY-plw,College Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/0GxxFZ...,https://www.yelp.com/biz/college-cafe-fort-col...,False,23,3.0,$,40.578299,-105.077356,647 S College Ave,Fort Collins,CO,80524,US,(970) 221-2696,1
27848,"Cheyenne, Wyoming",Turkish,zzeqVazbBK6-drbYWWLR9A,Petra Grill Mediterranean Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/LtyGPN...,https://www.yelp.com/biz/petra-grill-mediterra...,False,177,4.0,$$,40.574059,-105.100014,1335 W Elizabeth St,Fort Collins,CO,80521,US,(970) 568-8381,2
27849,"Cheyenne, Wyoming",Turkish,M2a9ML_cYBfjqwR9zxRYLg,Tikka Grill,https://s3-media2.fl.yelpcdn.com/bphoto/Xs6eqt...,https://www.yelp.com/biz/tikka-grill-fort-coll...,False,50,4.5,$,40.574839,-105.099282,1240 W Elizabeth St,Fort Collins,CO,80521,US,(970) 698-6022,1


In [23]:
#Create a heatmap based on price level
yelp_gmaps_2 = yelp_raw[['lat','long','price_level']]
yelp_gmaps_2 = yelp_gmaps_2[yelp_gmaps_2['price_level'] == 1]
yelp_gmaps_2.count()
heatmap_layer = gmaps.heatmap_layer(
    yelp_gmaps_2[['lat','long']],
    weights=yelp_gmaps_2['price_level'],
    max_intensity=50,
    point_radius=10.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
#Create a heatmap based on price level
yelp_gmaps_3 = yelp_raw[['lat','long','price_level']]
yelp_gmaps_3 = yelp_gmaps_3[yelp_gmaps_3['price_level'] == 3]
yelp_gmaps_3.count()
heatmap_layer = gmaps.heatmap_layer(
    yelp_gmaps_3[['lat','long']],
    weights=yelp_gmaps_3['price_level'],
    max_intensity=50,
    point_radius=10.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
#create a heat map to show most expensive restaurants
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    yelp_gmaps[['lat','long']],
    weights=yelp_gmaps['review_count'],
    max_intensity=100,
    point_radius=6.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
import gmaps
from config import gkey

fig = gmaps.figure(map_type='SATELLITE')

#generate some (latitude, longitude) pairs

specific_city = 'Albany, New York'
yelp_gb_cc_city = yelp_format_rd1[yelp_format_rd1['search_city'] == specific_city].groupby(['search_cuisine'])
variable2 = yelp_gb_cc_city['review_count'].sum()
variable2


search_cuisine
American         6933
Cajun            2956
Caribbean         518
Chinese          4296
French            995
German           1550
Greek            3446
Indian           2665
Italian          5191
Japanese         5136
Korean           4701
Lebanese          827
Mediterranean    2366
Mexican          5253
Moroccan         1036
Soul             2239
Spanish           857
Thai             5403
Turkish          1018
Vietnamese       3832
Name: review_count, dtype: int64

In [27]:
#Create yelp dataframe to compare concentration of businesses in cities
cities_df = pd.read_csv('../Project_1/cities.csv')
cities_df.dropna()
cities_df.head()

,Montgomery,Alabama
0,Juneau,Alaska
1,Phoenix,Arizona
2,Little Rock,Arkansas
3,Sacramento,California
4,Denver,Colorado


In [28]:
#store latitude and longitude in locations
#locations = cities_df[['lat', 'long']]
#weights = cities_df['name']
#fig = gmaps.figure()
#fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
#fig

In [29]:
#Create a data frame for the capitol cities, showing ratings and coordinates
capitalcities_df = pd.read_csv('../Project_1/cities.csv')
yelp_gmaps = yelp_format_rd1[yelp_format_rd1['search_city'] == specific_city]
yelp_gmaps = yelp_gmaps[['name','lat','long','review_count']]
yelp_gmaps.head()

,name,lat,long,review_count
17911,The Cuckoo's Nest,42.661730,-73.778150,249
17912,The Hollow Bar & Kitchen,42.652266,-73.750901,323
17913,Madison Pour House,42.666064,-73.791886,127
17914,The Bishop,42.652585,-73.751060,37
17915,The City Beer Hall,42.649550,-73.755030,505


In [30]:
#Create a heatmap of the restaurant distribution in US Capitals
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    yelp_gmaps[['lat','long']],
    weights=yelp_gmaps['price'],
    max_intensity=100,
    point_radius=6.0
)
fig.add_layer(heatmap_layer)
fig

KeyError: 'price'

In [ ]:
#Create a data frame that shows the review count and coordinate of all Mexican restaurants in Phoenix, Arizona
specific_city = 'Phoenix, Arizona'
specific_cuisine = 'Mexican'
yelp_gmaps = yelp_format_rd1[yelp_format_rd1['search_city'] == specific_city]
yelp_gmaps = yelp_gmaps[yelp_gmaps['search_cuisine'] == specific_cuisine]
yelp_gmaps = yelp_gmaps[['lat','long','review_count']]
yelp_gmaps.dropna()
yelp_gmaps.count()

In [ ]:
# Plot Heatmap
fig = gmaps.figure(map_type='HYBRID')

In [ ]:
# Plot Heatmap
#fig = gmaps.figure()

# Create heat layer
#heat_layer = gmaps.heatmap_layer(locations)

# Add layer
#fig.add_layer(heat_layer)

# Display figure
#fig

In [ ]:
# Plost as a Hybrid map
#fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
#heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
#                                 dissipating=False, max_intensity=10,
 #                                point_radius=1)

#fig.add_layer(heat_layer)
#fig

In [ ]:
#Note: Jorge: am working on creating a data frame for the city of Phoenix.  I am going to make a heatmap of the
    #restaurants in Phoenix, and then create a few heatmaps of different types of cuisines in the city.  

In [ ]:
yelp_gb_cuisinescount = yelp_format_rd1.groupby(['search_cuisine'])
yelp_gb_cuisinescount.count().head()

In [ ]:
yelp_gb_citiesavg = yelp_format_rd1.groupby(['search_city'])
yelp_gb_citiesavg.mean().head()

In [ ]:
yelp_gb_cuisinesavg = yelp_format_rd1.groupby(['search_cuisine'])
yelp_gb_cuisinesavg.mean().head()

In [ ]:
yelp_gb2_citiesavg = yelp_format_rd1.groupby(['search_city','search_cuisine'])
yelp_gb2_citiesavg.mean().head()

In [ ]:
yelp_gb2_cuisinesavg = yelp_format_rd1.groupby(['search_cuisine','search_city'])
yelp_gb2_cuisinesavg.mean().head()

In [ ]:
yelp_gb3_citiesavg = yelp_format_rd1.groupby(['search_city','search_cuisine'])
yelp_gb3_citiesavg.mean().head()

In [ ]:
yelp_gb3_cuisinesavg = yelp_format_rd1.groupby(['search_cuisine','search_city'])
yelp_gb3_cuisinesavg.mean().head()

In [ ]:
#This is where we are going to start charting, I have provided some exmaples for you. 

In [ ]:
chartme1 = yelp_gb_cuisinesavg['rating'].mean()
cuisine_chart = chartme1.plot(kind='bar', title='mean rating by cuisine')
cuisine_chart.set_xlabel("cuisine group")
cuisine_chart.set_ylabel("yelp rating")

plt.show()
plt.tight_layout()

In [ ]:
chartme2 = yelp_gb_citiesavg['rating'].mean()
cuisine_chart = chartme2.plot(kind='bar', title='mean rating by cuisine')
cuisine_chart.set_xlabel("yelp rating")
cuisine_chart.set_ylabel('our cities')

plt.show()
plt.tight_layout()